# Task 3 - Default Estimator and WoE Binning

This notebook is dedicated to the development of a credit scoring model through the construction of a default estimator and the application of Weight of Evidence (WoE) binning. The aim is to classify users into high-risk (bad) and low-risk (good) categories based on their likelihood of defaulting on credit, utilizing the RFMS (Recency, Frequency, Monetary value, Stability) formalism.

## Key Sections:

### 1. Default Estimator Construction
   - In this section, we analyze transaction behavior in the RFMS space to segment users. Visualization techniques are employed to establish boundaries and define a proxy for default, classifying users based on their RFMS scores.

### 2. Risk Classification
   - Users are labeled as high-risk (bad) or low-risk (good) using the default estimator built earlier. This classification helps in assigning risk profiles, which form the foundation of the credit scoring model.

### 3. Weight of Evidence (WoE) Binning
   - WoE binning is applied to transform both categorical and continuous features. This improves feature interpretability and enhances model performance by calculating WoE values for different feature bins.

---

This notebook guides you through the construction of a default estimator and the use of WoE binning to refine and optimize the credit scoring model. By following best practices in credit risk modeling, it ensures that the model is interpretable, reliable, and aligned with financial risk management standards.


# Importing Libraries

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))

In [2]:
from RFMS_score_WoE_binning import *
from plottings import *

* Loading the data

In [3]:
df = pd.read_csv('../data/processed_data.csv')

In [4]:
df.head()

,index,CustomerId,Amount,PricingStrategy,FraudResult,TotalTransactionAmount,AverageTransactionAmount,TotalTransactions,StdTransactionAmount,TransactionHour,...,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,0,CustomerId_4406,1000.0,2,0,0.557522,0.047184,-0.311831,-0.167524,2,...,0,0,0,0,0,0,0,0,1,0
1,1,CustomerId_4406,-20.0,2,0,0.557522,0.047184,-0.311831,-0.167524,2,...,0,0,0,0,0,0,0,1,0,0
2,2,CustomerId_4683,500.0,2,0,0.556944,0.047137,-0.444993,-0.201719,2,...,0,0,0,0,0,0,0,0,1,0
3,3,CustomerId_988,20000.0,2,0,0.558153,0.047749,-0.404020,-0.008737,3,...,0,0,0,0,0,1,0,0,1,0
4,4,CustomerId_988,-644.0,2,0,0.558153,0.047749,-0.404020,-0.008737,3,...,0,0,0,0,0,0,0,1,0,0


Create an instance of the RFMSCalculator

In [4]:
rfms_calculator = RFMSCalculator(df)

Calculate RFMS scores and get the results

In [5]:
df = rfms_calculator.calculate_rfms()

Starting RFMS calculation
Calculating Recency
Recency calculation completed
Calculating Frequency
Frequency calculation completed
Calculating Monetary Value
Monetary value calculation completed
Merged RFMS DataFrame columns: ['CustomerId', 'TotalTransactions', 'AverageTransactionAmount', 'StdTransactionAmount_x', 'TotalTransactionAmount', 'MonetaryAverageTransactionAmount', 'StdTransactionAmount_y']
        CustomerId  TotalTransactions  AverageTransactionAmount  \
0     CustomerId_1                  1             -10000.000000   
1    CustomerId_10                  1             -10000.000000   
2  CustomerId_1001                  5               4000.000000   
3  CustomerId_1002                 11                384.090909   
4  CustomerId_1003                  6               3333.333333   

   StdTransactionAmount_x  TotalTransactionAmount  \
0                     NaN                -10000.0   
1                     NaN                -10000.0   
2             6558.963333          

# Assign Good/Bad Labels based on RFMS formalism

Create an instance of the Labeling class to assign good/bad labels

In [6]:
labeler = Labeling(df)


In [7]:
df_labeled = labeler.assign_good_bad_labels()

Assigning Good/Bad labels
Good/Bad labels assigned


In [12]:
df.columns

Index(['CustomerId', 'TotalTransactions', 'AverageTransactionAmount',
       'StdTransactionAmount_x', 'TotalTransactionAmount',
       'MonetaryAverageTransactionAmount', 'StdTransactionAmount_y',
       'RFMS_Score', 'RFMS_Label', 'User_Label'],
      dtype='object')

In [13]:
# Perform WoE Binning for the 'Monetary' Feature
woe_binner = WoEBinning(df_labeled, 'MonetaryAverageTransactionAmount', 'User_Label')


In [23]:
woe_results = woe_binner.calculate_woe()

c:\Users\a\Documents\GitHub\Bati-Bank-Credit-Scoring-Model\scripts\RFMS_score_WoE_binning.py:165: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  woe_df = self.df.groupby(binned_df)[self.target].value_counts(normalize=False).unstack()


In [15]:
# Display WoE Binning Results
print(woe_results)

User_Label                        Good_Dist  Bad_Dist       WoE
MonetaryAverageTransactionAmount                               
(-434026.821, 477682.114]          0.986451       1.0 -0.013641
(477682.114, 1380364.229]          0.010943       0.0       inf
(1380364.229, 2283046.343]         0.002084       0.0       inf
(2283046.343, 3185728.457]         0.000000       0.0       NaN
(3185728.457, 4088410.571]         0.000000       0.0       NaN
(4088410.571, 4991092.686]         0.000000       0.0       NaN
(4991092.686, 5893774.8]           0.000000       0.0       NaN
(5893774.8, 6796456.914]           0.000000       0.0       NaN
(6796456.914, 7699139.029]         0.000000       0.0       NaN
(7699139.029, 8601821.143]         0.000521       0.0       inf


In [24]:
# Split Data into Train/Test for Validation
train_df, test_df = train_test_split(df_labeled, test_size=0.2, random_state=42, stratify=df_labeled['User_Label'])


In [17]:
# Validate Default Estimator (AUC, ROC)
y_train = np.where(train_df['User_Label'] == 'Good', 0, 1)
y_test = np.where(test_df['User_Label'] == 'Good', 0, 1)
y_train_pred = train_df['RFMS_Label']
y_test_pred = test_df['RFMS_Label']

In [18]:
auc_train = roc_auc_score(y_train, y_train_pred)
auc_test = roc_auc_score(y_test, y_test_pred)

In [19]:
print(f"Train AUC: {auc_train:.2f}")
print(f"Test AUC: {auc_test:.2f}")

Train AUC: 1.00
Test AUC: 1.00


In [20]:
# Generate Classification Report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       384
           1       1.00      1.00      1.00       365

    accuracy                           1.00       749
   macro avg       1.00      1.00      1.00       749
weighted avg       1.00      1.00      1.00       749

